In [ ]:
# Useful starting lines
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.linear_model as sk
import re
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np


def build_poly(x, degree):
    """polynomial basis functions for input data x, for j=0 up to j=degree."""
    poly = np.ones((len(x), 1))
    for deg in range(1, degree+1):
        poly = np.c_[poly, np.power(x, deg)]
    return poly

In [ ]:
data = open('data/test_data_clean_v1.txt',encoding='utf-8').readlines()
idx = np.zeros(np.shape(data)[0])
tweets = ["" for a in range(0,np.shape(data)[0])]
for i in range(0,np.shape(data)[0]):
    idx[i] = i
    tweets[i] = data[i]
print(idx.shape)
print(tweets.shape)

In [ ]:
    pos_train = open('data/pos_train.txt').readlines()
    neg_train = open('data/neg_train.txt').readlines()
    embeddings = np.load('data/embeddings.npy')

    
    #count number of word/tweet and store it for both positive set and negative set
    word_nbr_per_tweet_pos = np.zeros(np.shape(pos_train)[0])
    for j in range(0,np.shape(pos_train)[0]):
        tweet = pos_train[j]
        size = len(tweet.split())
        word_nbr_per_tweet_pos[j] = size
    
    word_nbr_per_tweet_neg = np.zeros(np.shape(neg_train)[0])
    for j in range(0,np.shape(neg_train)[0]):
        tweet = neg_train[j]
        size = len(tweet.split())
        word_nbr_per_tweet_neg[j] = size
    
    print("counting ended")
    
    i=0
    pos_mask = np.zeros(np.shape(embeddings)[1]+1)
    pos_mask[0] +=1
    #adding 1 at start : this is target (1 is for happy emoji, 0 or -1 for sad face)
    training_set_pos = np.zeros(((np.shape(pos_train)[0],np.shape(embeddings)[1]+1))) + pos_mask
    training_set_neg = np.zeros(((np.shape(neg_train)[0],np.shape(embeddings)[1]+1)))
    vocab = open('data/vocab_cut.txt')
    #for each word, search if it is in pos_train or neg_train
    prevWord =""
    for word_ in vocab:
        word = word_.split("\n")[0]
        #if charac is ecaped, a special regex is used this is why there is a boolean
        if(re.escape(word) != word):
            word= re.escape(word)
        current_emb = embeddings[i]
        for j in range(0,np.shape(pos_train)[0]):
            #if yes, add its embeddings.
            #if word in pos_train[j]:
            if(prevWord != word):
                print(word)
                prevWord=word
            if re.search(r"(?:(?:(?:\b)|^)"+word+"(?:(?=\b)|$)|(?:^|(?:\s))"+word+"(?:$|(?=\s)))",pos_train[j]):
                count = re.findall(r"(?:(?:(?:\b)|^)"+word+"(?:(?=\b)|$)|(?:^|(?:\s))"+word+"(?:$|(?=\s)))",pos_train[j])
                if(j < 20):
                    print(word,pos_train[j],len(count))
                training_set_pos[j,1:np.shape(embeddings)[1]+1] += (len(count)*current_emb)
        for j in range(0,np.shape(neg_train)[0]):
            #if word in neg_train[j]:
            if re.search(r"(?:(?:(?:\b)|^)"+word+"(?:(?=\b)|$)|(?:^|(?:\s))"+word+"(?:$|(?=\s)))",neg_train[j]):
                count = re.findall(r"(?:(?:(?:\b)|^)"+word+"(?:(?=\b)|$)|(?:^|(?:\s))"+word+"(?:$|(?=\s)))",neg_train[j])
                training_set_neg[j,1:np.shape(embeddings)[1]+1] += (len(count)*current_emb)
        i+=1
        if(i%5000 ==0):
            print("5000 done")
    #then divide by number of words (averaging word vector over all words of the tweet)
    for i in range(0,np.shape(embeddings)[1]):
        training_set_pos[:,i+1] = training_set_pos[:,i+1]/word_nbr_per_tweet_pos
        training_set_neg[:,i+1] = training_set_neg[:,i+1]/word_nbr_per_tweet_neg


In [ ]:
#!/usr/bin/env python3
from scipy.sparse import *
import csv
import numpy as np
import pandas as pd
import sklearn.linear_model as sk
import sklearn.model_selection as ms
from sklearn import svm
import pickle
import random
import re
import string

def construct_vectors(data,set_to_fill,vocab,embeddings):
    list_auxiliarry_pos = ["must","need","should","may","might","can","could","shall","would","will"]
    list_auxiliarry_neg = ["won't","shouldn't","not","can't","couldn't","wouldn't"]
    counter = lambda l1, l2: len(list(filter(lambda c: c in l2, l1)))
    
    
    for j in range(0,np.shape(data)[0]):
        num_punctu = counter(data[j],string.punctuation)
        list_word = data[j].split()
        average = 0
        num3point = 0
        num_aux_pos =0
        num_aux_neg =0
        for i in list_word:
            average+=len(i)
            if(i=="..."):
                num3point+=1
            if(i in list_auxiliarry_pos):
                num_aux_pos+=1
            if(i in list_auxiliarry_neg):
                num_aux_neg+=1
            idx = vocab.get(i,-1)
            if(idx>=0):
                set_to_fill[j,1:np.shape(embeddings)[1]+1] += embeddings[idx]
        set_to_fill[j,1:np.shape(embeddings)[1]+1] = set_to_fill[j,1:np.shape(embeddings)[1]+1]/len(list_word)
        set_to_fill[j,np.shape(embeddings)[1]+1] = len(list_word) #add the # word
        set_to_fill[j,np.shape(embeddings)[1]+2] = num_punctu #add the # punctuation
        set_to_fill[j,np.shape(embeddings)[1]+3] = average/len(list_word) #add length of word in average
        set_to_fill[j,np.shape(embeddings)[1]+4] = num_aux_pos #word in a list of auxilarry
        set_to_fill[j,np.shape(embeddings)[1]+5] = num_aux_neg #word in a list of negative aux
        set_to_fill[j,np.shape(embeddings)[1]+6] = num3point #number of ...
    return set_to_fill
def construct_features():
    '''
    construct a feature representation of each training tweet 
    (by averaging the word vectors over all words of the tweet).
    '''
    #Load the training tweets and the built GloVe word embeddings
    additional_features = 6

    pos_train = open('data/pos_train.txt').readlines()
    neg_train = open('data/neg_train.txt').readlines()
    embeddings = np.load('data/embeddings.npy')
    with open('data/vocab.pkl', 'rb') as f:
        vocab = pickle.load(f)

    pos_mask = np.zeros(np.shape(embeddings)[1]+1+additional_features)
    pos_mask[0] +=1
    #adding 1 at start : this is target (1 is for happy emoji, 0 or -1 for sad face)
    #will add 3 features , number of word , average length of words, and #punctuation
    training_set_pos = np.zeros(((np.shape(pos_train)[0],np.shape(embeddings)[1]+1+additional_features))) + pos_mask
    training_set_neg = np.zeros(((np.shape(neg_train)[0],np.shape(embeddings)[1]+1+additional_features)))
    #for each word, search if it is in pos_train or neg_train
    
    training_set_pos = construct_vectors(pos_train,training_set_pos,vocab,embeddings) #look at method above
    training_set_neg = construct_vectors(neg_train,training_set_neg,vocab,embeddings)
    np.save('data/trainingset_pos', training_set_pos)
    np.save('data/trainingset_neg', training_set_neg)
    
def create_csv_submission(ids, y_pred, name):
    """
    Creates an output file in csv format for submission to kaggle
    Arguments: ids (event ids associated with each prediction)
               y_pred (predicted class labels)
               name (string name of .csv output file to be created)
    """
    with open(name, 'w') as csvfile:
        fieldnames = ['Id', 'Prediction']
        writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
        writer.writeheader()
        for r1, r2 in zip(ids, y_pred):
            writer.writerow({'Id':int(r1),'Prediction':int(r2)})

def predict_labels(flag=".npy"):
    #Load the training set
    path_neg = str("data/trainingset_neg"+flag)
    path_pos = str("data/trainingset_pos"+flag)
    ts_neg = np.load(path_neg)
    ts_pos = np.load(path_pos)    
    #Train a Linear Classifier: Train a linear classifier (e.g. logistic regression or SVM) on your constructed 
    #features, using the scikit learn library, or your own code from the earlier labs. Recall that the labels 
    #indicate if a tweet used to contain a :) or :( smiley.
    training_set = np.concatenate((ts_neg,ts_pos))
    y = training_set[:,0]
    X = training_set[:,1:np.shape(training_set)[1]]
    X = build_poly(X,2)
    #Now we load and predict the data
    data = np.genfromtxt('data/test_data.txt', delimiter="\n",dtype=str)    
    idx = np.zeros(np.shape(data)[0])
    tweets = ["" for a in range(0,np.shape(data)[0])]
    for i in range(0,np.shape(data)[0]):
        spliter = data[i].split(",")
        idx[i] = spliter[0]
        tweet = spliter[1]
        for j in range(2,np.shape(spliter)[0]):
            tweet = tweet+","+spliter[j]
        tweets[i] = tweet
    
    #Construct the logistic regressor
    LR = sk.LogisticRegressionCV()
    #LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, 
    #class_weight=None, random_state=None, solver='liblinear', max_iter=100, multi_class='ovr', verbose=0, 
    #warm_start=False, n_jobs=1)[source]¶
    #http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
    #train the logistic regressor
    kf = ms.KFold(n_splits=3,shuffle=True)
    for train_idx, test_idx in kf.split(X):
        train_set = X[train_idx]
        test_set = X[test_idx]
        train_target = y[train_idx]
        test_target = y[test_idx]    
        LR.fit(train_set,train_target)
        predictions_temp = LR.predict(test_set)
        print(predictions_temp.shape)
        print(test_target.shape)        
        error = np.sum(np.power(predictions_temp-test_target,2))/np.shape(predictions_temp)[0]
        print("Yet, error is",error)
    LR.fit(X,y)
    
    #And now, predict the results
    topredict = construct_features_for_test_set(tweets)
    predictions = LR.predict(topredict)
    #Construct the submission
    predictions = predictions*2-1
    create_csv_submission(idx,predictions,"submission.csv")

def construct_features_for_test_set(test_set_tweet):
    embeddings = np.load('data/embeddings.npy')
    with open('data/vocab.pkl', 'rb') as f:
        vocab = pickle.load(f)
    additional_features = 6
    list_auxiliarry_pos = ["must","need","should","may","might","can","could","shall","would","will"]
    list_auxiliarry_neg = ["won't","shouldn't","not","can't","couldn't","wouldn't"]
    test_set = np.zeros((np.shape(test_set_tweet)[0],np.shape(embeddings)[1]+additional_features))
    #for each word, search if it is in a tweet
    for j in range(0,np.shape(test_set)[0]):
        list_word = test_set_tweet[j].split()
        divider = 0
        average = 0
        num3point = 0
        num_aux_pos = 0
        num_aux_neg = 0
        counter = lambda l1, l2: len(list(filter(lambda c: c in l2, l1)))
        num_punctu = counter(test_set_tweet[j],string.punctuation)
        for i in list_word:
            idx = vocab.get(i,-1)
            average+=len(i)
            if(i=="..."):
                num3point+=1
            if(i in list_auxiliarry_pos):
                num_aux_pos+=1
            if(i in list_auxiliarry_neg):
                num_aux_neg+=1
            if(idx>=0):
                divider+=1
                test_set[j,:np.shape(embeddings)[1]] += embeddings[idx]
        if(divider >0):
            test_set[j,:np.shape(embeddings)[1]] = test_set[j,:np.shape(embeddings)[1]]/divider
        test_set[j,np.shape(embeddings)[1]] = len(list_word) #add the # word
        test_set[j,np.shape(embeddings)[1]+1] = num_punctu #add the # punctuation
        if(len(list_word) >0):
            test_set[j,np.shape(embeddings)[1]+2] = average/len(list_word)#add length of word in average
        else : 
            test_set[j,np.shape(embeddings)[1]+2] = 0
        test_set[j,np.shape(embeddings)[1]+3] = num_aux_pos
        test_set[j,np.shape(embeddings)[1]+4] = num_aux_neg
        test_set[j,np.shape(embeddings)[1]+5] = num3point
    #then divide by number of words (averaging word vector over all words of the tweet)
    return test_set
construct_features()
predict_labels()


In [ ]:
from nltk.stem import PorterStemmer
global_stemmer = PorterStemmer()
 
class StemmingHelper(object):
    """
    Class to aid the stemming process - from word to stemmed form,
    and vice versa.
    The 'original' form of a stemmed word will be returned as the
    form in which its been used the most number of times in the text.
    """ 
    #This reverse lookup will remember the original forms of the stemmed
    #words
    word_lookup = {}
    @classmethod
    def stem(cls, word):
        """
        Stems a word and updates the reverse lookup.
        """
        #Stem the word
        stemmed = global_stemmer.stem(word)
 
        #Update the word lookup
        if stemmed not in cls.word_lookup:
            cls.word_lookup[stemmed] = {}
        cls.word_lookup[stemmed][word] = (
            cls.word_lookup[stemmed].get(word, 0) + 1)
 
        return stemmed
 
    @classmethod
    def original_form(cls, word):
        """
        Returns original form of a word given the stemmed version,
        as stored in the word lookup.
        """
 
        if word in cls.word_lookup:
            return max(cls.word_lookup[word].keys(),
                       key=lambda x: cls.word_lookup[word][x])
        else:
            return word

In [ ]:
import re
from stop_words import get_stop_words

stop = get_stop_words('english')
pattern = '|'.join(stop)
prog = re.compile(pattern, re.IGNORECASE)  # re.IGNORECASE to catch both 'La' and 'la' 

input_file_location = 'data/full10_tweets.txt'
output_file_location = 'data/full10_tweets_stemmed.txt'

with open(input_file_location, 'r') as fin:
    with open(output_file_location, 'w') as fout:
        for l in fin:
            m = l.split()  # l.strip() to remove leading/trailing whitespace
            for i in m :
                if(StemmingHelper.stem(i) != i):
                    fout.write('%s ' % StemmingHelper.stem(i))
                else:
                     fout.write('%s ' %i)
            fout.write('\n')
input_file_location = 'data/neg_train.txt'
output_file_location = 'data/neg_train_stemmed.txt'

with open(input_file_location, 'r') as fin:
    with open(output_file_location, 'w') as fout:
        for l in fin:
            m = l.split()  # l.strip() to remove leading/trailing whitespace
            for i in m :
                if(StemmingHelper.stem(i) != i):
                    fout.write('%s ' % StemmingHelper.stem(i))
                else:
                     fout.write('%s ' %i)
            fout.write('\n')
input_file_location = 'data/pos_train.txt'
output_file_location = 'data/pos_train_stemmed.txt'

with open(input_file_location, 'r') as fin:
    with open(output_file_location, 'w') as fout:
        for l in fin:
            m = l.split()  # l.strip() to remove leading/trailing whitespace
            for i in m :
                if(StemmingHelper.stem(i) != i):
                    fout.write('%s ' % StemmingHelper.stem(i))
                else:
                     fout.write('%s ' %i)
            fout.write('\n')
            
input_file_location = 'data/test_data.txt'
output_file_location = 'data/test_data_stemmed.txt'

with open(input_file_location, 'r') as fin:
    with open(output_file_location, 'w') as fout:
        for l in fin:
            m = l.split()  # l.strip() to remove leading/trailing whitespace
            for i in m :
                if(StemmingHelper.stem(i) != i):
                    fout.write('%s ' % StemmingHelper.stem(i))
                else:
                     fout.write('%s ' %i)
            fout.write('\n')

In [ ]:
    from gensim.models import word2vec

    size = 200
    window = 8
    sentences = word2vec.LineSentence('data/train_clean_v1_1.txt')
    model = word2vec.Word2Vec(sentences, size=size,window =window)

In [ ]:
    from gensim.models import word2vec
    import string
    
    list_auxiliarry_pos = ["must","need","should","may","might","can","could","shall","would","will"]
    list_auxiliarry_neg = ["won't","shouldn't","not","can't","couldn't","wouldn't"]
    counter = lambda l1, l2: len(list(filter(lambda c: c in l2, l1))) #Used later to count number fo punctuation
    additional_features = 6
    
    pos_train = open('data/pos_clean_v1_1.txt').readlines()
    lengt = size
    pos_mask = np.zeros(lengt+1+additional_features)
    pos_mask[0] +=1
    #adding 1 at start : this is target (1 is for happy emoji, 0 or -1 for sad face)
    #will add 3 features , number of word , average length of words, and #punctuation
    training_set_pos = np.zeros(((np.shape(pos_train)[0],lengt+1+additional_features))) + pos_mask
    #for each word, search if it is in pos_train or neg_train
    for j in range(0,np.shape(pos_train)[0]):
        list_word = pos_train[j].split()
        num_punctu = counter(pos_train[j],string.punctuation)
        divider = 0
        average = 0
        num3point = 0
        num_aux_pos =0
        num_aux_neg =0
        for i in list_word:
            average+=len(i)
            if(i=="..."): 
                num3point+=1
            if(i in list_auxiliarry_pos):
                num_aux_pos+=1
            if(i in list_auxiliarry_neg):
                num_aux_neg+=1
            if(i in model):
                divider+=1
                training_set_pos[j,1:lengt+1] += model[i]
        if(divider>0):
            training_set_pos[j,1:lengt+1] = (training_set_pos[j,1:lengt+1]/divider)
        training_set_pos[j,lengt+1] = len(list_word) #add the # word
        training_set_pos[j,lengt+2] = num_punctu #add the # punctuation
        if(len(list_word)>0):
            training_set_pos[j,lengt+3] = average/len(list_word) #add length of word in average
        else :
            training_set_pos[j,lengt+3] = 0
        training_set_pos[j,lengt+4] = num_aux_pos #word in a list of auxilarry
        training_set_pos[j,lengt+5] = num_aux_neg #word in a list of negative aux
        training_set_pos[j,lengt+6] = num3point #number of ...
    
    neg_train = open('data/neg_clean_v1_1.txt',encoding='utf-8').readlines()
    training_set_neg = np.zeros(((np.shape(neg_train)[0],lengt+1+additional_features)))
    #for each word, search if it is in pos_train or neg_train
    for j in range(0,np.shape(neg_train)[0]):
        num_punctu = counter(neg_train[j],string.punctuation)
        list_word = neg_train[j].split()
        divider = 0
        average = 0
        num3point = 0
        num_aux_pos =0
        num_aux_neg =0
        for i in list_word:
            average+=len(i)
            if(i=="..."): 
                num3point+=1
            if(i in list_auxiliarry_pos):
                num_aux_pos+=1
            if(i in list_auxiliarry_neg):
                num_aux_neg+=1
            if(i in model):
                divider+=1
                training_set_neg[j,1:lengt+1] += model[i]
        if(divider>0):
            training_set_neg[j,1:lengt+1] = (training_set_neg[j,1:lengt+1]/divider)
        training_set_neg[j,lengt+1] = len(list_word) #add the # word
        training_set_neg[j,lengt+2] = num_punctu #add the # punctuation
        if(len(list_word)>0):
            training_set_neg[j,lengt+3] = average/len(list_word) #add length of word in average
        else:
            training_set_neg[j,lengt+3] = 0
        training_set_neg[j,lengt+4] = num_aux_pos #word in a list of auxilarry
        training_set_neg[j,lengt+5] = num_aux_neg #word in a list of negative aux
        training_set_neg[j,lengt+6] = num3point #number of ...
    np.save('data/trainingsetneural_pos', training_set_pos)
    np.save('data/trainingsetneural_neg', training_set_neg)

In [ ]:
from scipy.sparse import *
import csv
import numpy as np
import pandas as pd
import sklearn.linear_model as sk
import sklearn.model_selection as ms
from sklearn import svm
import pickle
import random
import re
import string
def create_csv_submission(ids, y_pred, name):
    """
    Creates an output file in csv format for submission to kaggle
    Arguments: ids (event ids associated with each prediction)
               y_pred (predicted class labels)
               name (string name of .csv output file to be created)
    """
    with open(name, 'w') as csvfile:
        fieldnames = ['Id', 'Prediction']
        writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
        writer.writeheader()
        for r1, r2 in zip(ids, y_pred):
            writer.writerow({'Id':int(r1),'Prediction':int(r2)})

def predict_labels(flag=".npy"):
    #Load the training set
    path_neg = str("data/trainingsetneural_neg"+flag)
    path_pos = str("data/trainingsetneural_pos"+flag)
    ts_neg = np.load(path_neg)
    ts_pos = np.load(path_pos)
    #Train a Linear Classifier: Train a linear classifier (e.g. logistic regression or SVM) on your constructed 
    #features, using the scikit learn library, or your own code from the earlier labs. Recall that the labels 
    #indicate if a tweet used to contain a :) or :( smiley.
    training_set = np.concatenate((ts_neg,ts_pos))
    y = training_set[:,0]
    X = training_set[:,1:np.shape(training_set)[1]]
    #Now we load and predict the data
    data = open('data/test_clean_v1_1.txt',encoding='utf-8').readlines()  
    idx = np.zeros(np.shape(data)[0])
    tweets = ["" for a in range(0,np.shape(data)[0])]
    for i in range(0,np.shape(data)[0]):
        idx[i] =(i+1)
        tweets[i] = data[i]
    
    #Construct the logistic regressor
    LR = sk.LogisticRegressionCV()
    #LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, 
    #class_weight=None, random_state=None, solver='liblinear', max_iter=100, multi_class='ovr', verbose=0, 
    #warm_start=False, n_jobs=1)[source]¶
    #http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
    #train the logistic regressor
    kf = ms.KFold(n_splits=3,shuffle=True)
    X_poly = X
    for train_idx, test_idx in kf.split(X_poly):
        train_set = X_poly[train_idx]
        test_set = X_poly[test_idx]
        train_target = y[train_idx]
        test_target = y[test_idx]    
        LR.fit(train_set,train_target)
        predictions_temp = LR.predict(test_set)
        print(predictions_temp.shape)
        print(test_target.shape)        
        error = np.sum(np.power(predictions_temp-test_target,2))/np.shape(predictions_temp)[0]
        print("Yet, error is",error)
    LR.fit(X_poly,y)
    
    #And now, predict the results
    topredict = construct_features_for_test_set(tweets)
    predictions = LR.predict(topredict)
    #Construct the submission
    predictions = predictions*2-1
    create_csv_submission(idx,predictions,"submission.csv")
    
    #for each word, search if it is in a tweet
def construct_features_for_test_set(test_set_tweet):
    list_auxiliarry_pos = ["must","need","should","may","might","can","could","shall","would","will"]
    list_auxiliarry_neg = ["won't","shouldn't","not","can't","couldn't","wouldn't"]
    counter = lambda l1, l2: len(list(filter(lambda c: c in l2, l1))) #Used later to count number fo punctuation
    
    additional_features = 6
    test_set = np.zeros((np.shape(test_set_tweet)[0],lengt+additional_features))
    for j in range(0,np.shape(test_set)[0]):
        num_punctu = counter(test_set_tweet[j],string.punctuation)
        list_word = test_set_tweet[j].split()
        divider = 0
        average = 0
        num3point = 0
        num_aux_pos =0
        num_aux_neg =0
        for i in list_word:
            average+=len(i)
            if(i=="..."): 
                num3point+=1
            if(i in list_auxiliarry_pos):
                num_aux_pos+=1
            if(i in list_auxiliarry_neg):
                num_aux_neg+=1
            if(i in model):
                divider+=1
                test_set[j,:lengt] += model[i]
        if(divider>0):
            test_set[j,:lengt] = (test_set[j,:lengt]/divider)
        test_set[j,lengt] = len(list_word) #add the # word
        test_set[j,lengt+1] = num_punctu #add the # punctuation
        if(len(list_word) >0):
            test_set[j,lengt+2] = average/len(list_word)#add length of word in average
        else : 
            test_set[j,lengt+2] = 0
        test_set[j,lengt+3] = num_aux_pos #word in a list of auxilarry
        test_set[j,lengt+4] = num_aux_neg #word in a list of negative aux
        test_set[j,lengt+5] = num3point #number of ...
    return test_set
predict_labels()

In [ ]:
with open("data/test.txt", 'r+') as f:
    lines = f.readlines()
    for j in range(np.shape(lines)[0]):
        list_word = lines[j].split()
        sentence = ""
        print(list_word)
        f.seek(0)
        for i in list_word:
            print(i)
            if(i=="try" or i=="out"):
                sentence+=(i+" ")
        print(sentence)
        f.truncate()
        f.write(sentence)

In [ ]:
alllosses = []

In [ ]:
#!/usr/bin/env python3
from scipy.sparse import *
import csv
import numpy as np
import pandas as pd
import sklearn.linear_model as sk
import sklearn.model_selection as ms
from sklearn import svm
import pickle
import random
import re
import string

#This method construct the features of each tweets
def construct_vectors(data,set_to_fill,vocab,embeddings):
    list_auxiliarry_pos = ["must","need","should","may","might","can","could","shall","would","will"]
    list_auxiliarry_neg = ["won't","shouldn't","not","can't","couldn't","wouldn't"]
    counter = lambda l1, l2: len(list(filter(lambda c: c in l2, l1))) #Used later to count number fo punctuation
    
    
    for j in range(0,np.shape(data)[0]):
        num_punctu = counter(data[j],string.punctuation)
        list_word = data[j].split()
        average = 0
        num3point = 0
        num_aux_pos =0
        num_aux_neg =0
        num_user = 0
        for i in list_word:
            average+=len(i)
            if(i=="<user>"):
                num_user+=1
            if(i=="..."): 
                num3point+=1
            #if(i in list_auxiliarry_pos):
               #num_aux_pos+=1
            if(i in list_auxiliarry_neg):
                num_aux_neg+=1
            idx = vocab.get(i,-1)
            if(idx>=0):
                set_to_fill[j,1:np.shape(embeddings)[1]+1] += embeddings[idx]
        set_to_fill[j,1:np.shape(embeddings)[1]+1] = set_to_fill[j,1:np.shape(embeddings)[1]+1]/len(list_word) #put the average of the word vector in dim [1,21]
        set_to_fill[j,np.shape(embeddings)[1]+1] = len(list_word) #add the # word
        set_to_fill[j,np.shape(embeddings)[1]+2] = num_punctu #add the # punctuation
        set_to_fill[j,np.shape(embeddings)[1]+3] = average/len(list_word) #add length of word in average
        set_to_fill[j,np.shape(embeddings)[1]+4] = num_aux_pos #word in a list of auxilarry
        set_to_fill[j,np.shape(embeddings)[1]+5] = num_aux_neg #word in a list of negative aux
        set_to_fill[j,np.shape(embeddings)[1]+6] = num3point #number of ...
        set_to_fill[j,np.shape(embeddings)[1]+7] = num3point #number of ...
    return set_to_fill
def construct_features():
    '''
    construct a feature representation of each training tweet 
    (by averaging the word vectors over all words of the tweet).
    '''
    #Load the training tweets and the built GloVe word embeddings
    additional_features = 7

    pos_train = open('data/pos_train.txt').readlines()
    neg_train = open('data/neg_train.txt').readlines()
    embeddings = np.load('data/embeddings.npy')
    with open('data/vocab.pkl', 'rb') as f:
        vocab = pickle.load(f)

    pos_mask = np.zeros(np.shape(embeddings)[1]+1+additional_features)
    pos_mask[0] +=1
    #adding 1 at start : this is target (1 is for happy emoji, 0 or -1 for sad face)
    #will add 3 features , number of word , average length of words, and #punctuation
    training_set_pos = np.zeros(((np.shape(pos_train)[0],np.shape(embeddings)[1]+1+additional_features))) + pos_mask
    training_set_neg = np.zeros(((np.shape(neg_train)[0],np.shape(embeddings)[1]+1+additional_features)))
    
    training_set_pos = construct_vectors(pos_train,training_set_pos,vocab,embeddings) #Construct the two training set
    training_set_neg = construct_vectors(neg_train,training_set_neg,vocab,embeddings)
    np.save('data/trainingset_pos', training_set_pos)
    np.save('data/trainingset_neg', training_set_neg)
    
def create_csv_submission(ids, y_pred, name):
    """
    Creates an output file in csv format for submission to kaggle
    Arguments: ids (event ids associated with each prediction)
               y_pred (predicted class labels)
               name (string name of .csv output file to be created)
    """
    with open(name, 'w') as csvfile:
        fieldnames = ['Id', 'Prediction']
        writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
        writer.writeheader()
        for r1, r2 in zip(ids, y_pred):
            writer.writerow({'Id':int(r1),'Prediction':int(r2)})

def predict_labels(flag=".npy"):
    #Load the training set
    path_neg = str("data/trainingset_neg"+flag)
    path_pos = str("data/trainingset_pos"+flag)
    ts_neg = np.load(path_neg)
    ts_pos = np.load(path_pos)    
    #Train a Linear Classifier: Train a linear classifier (e.g. logistic regression or SVM) on your constructed 
    #features, using the scikit learn library, or your own code from the earlier labs. Recall that the labels 
    #indicate if a tweet used to contain a :) or :( smiley.
    training_set = np.concatenate((ts_neg,ts_pos))
    y = training_set[:,0]
    X = training_set[:,1:np.shape(training_set)[1]]
    X = build_poly(X,2)
    #Now we load and predict the data
    
    #Construct the logistic regressor
    LR = sk.LogisticRegressionCV()
    #LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, 
    #class_weight=None, random_state=None, solver='liblinear', max_iter=100, multi_class='ovr', verbose=0, 
    #warm_start=False, n_jobs=1)[source]¶
    #http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
    #train the logistic regressor
    kf = ms.KFold(n_splits=3,shuffle=True)
    summing = 0
    for train_idx, test_idx in kf.split(X):
        train_set = X[train_idx]
        test_set = X[test_idx]
        train_target = y[train_idx]
        test_target = y[test_idx]    
        LR.fit(train_set,train_target)
        predictions_temp = LR.predict(test_set)
        print(predictions_temp.shape)
        print(test_target.shape)        
        error = np.sum(np.power(predictions_temp-test_target,2))/np.shape(predictions_temp)[0]
        summing +=error
        print("Yet, error is",error)
    
        

def construct_features_for_test_set(test_set_tweet):
    embeddings = np.load('data/embeddings.npy')
    with open('data/vocab.pkl', 'rb') as f:
        vocab = pickle.load(f)
    additional_features = 6
    list_auxiliarry_pos = ["must","need","should","may","might","can","could","shall","would","will"]
    list_auxiliarry_neg = ["won't","shouldn't","not","can't","couldn't","wouldn't"]
    test_set = np.zeros((np.shape(test_set_tweet)[0],np.shape(embeddings)[1]+additional_features))
    #Do the same as the training set, but do not have the label at first , so method is a little different
    for j in range(0,np.shape(test_set)[0]):
        list_word = test_set_tweet[j].split()
        divider = 0
        average = 0
        num3point = 0
        num_aux_pos = 0
        num_aux_neg = 0
        counter = lambda l1, l2: len(list(filter(lambda c: c in l2, l1)))
        num_punctu = counter(test_set_tweet[j],string.punctuation)
        for i in list_word:
            idx = vocab.get(i,-1)
            average+=len(i)
            if(i=="..."):
                num3point+=1
            #if(i in list_auxiliarry_pos):
                #num_aux_pos+=1
            if(i in list_auxiliarry_neg):
                num_aux_neg+=1
            if(idx>=0):
                divider+=1
                test_set[j,:np.shape(embeddings)[1]] += embeddings[idx]
        if(divider >0):
            test_set[j,:np.shape(embeddings)[1]] = test_set[j,:np.shape(embeddings)[1]]/divider
        test_set[j,np.shape(embeddings)[1]] = len(list_word) #add the # word
        test_set[j,np.shape(embeddings)[1]+1] = num_punctu #add the # punctuation
        if(len(list_word) >0):
            test_set[j,np.shape(embeddings)[1]+2] = average/len(list_word)#add length of word in average
        else : 
            test_set[j,np.shape(embeddings)[1]+2] = 0
        test_set[j,np.shape(embeddings)[1]+3] = num_aux_pos
        test_set[j,np.shape(embeddings)[1]+4] = num_aux_neg
        test_set[j,np.shape(embeddings)[1]+5] = num3point
    #then divide by number of words (averaging word vector over all words of the tweet)
    return test_set
construct_features()
predict_labels()


In [ ]:
import matplotlib.pyplot as plt
print(alllosses)
plt.plot(alllosses)

In [ ]:
data = open('data/test_clean_v1_1.txt',encoding='utf-8').readlines()
print(data[3])

In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import re
import datetime
from tensorflow.contrib import learn

class TextCNN(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    """
    def __init__(
      self, sequence_length, num_classes, vocab_size,
      embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.0):

        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)

        # Embedding layer
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            W = tf.Variable(
                tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),
                name="W")
            self.embedded_chars = tf.nn.embedding_lookup(W, self.input_x)
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)

        # Create a convolution + maxpool layer for each filter size
        pooled_outputs = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                # Maxpooling over the outputs
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled_outputs.append(pooled)

        # Combine all the pooled features
        num_filters_total = num_filters * len(filter_sizes)
        self.h_pool = tf.concat(3, pooled_outputs)
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])

        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")

        # CalculateMean cross-entropy loss
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(self.scores, self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        # Accuracy
        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")

In [2]:
import itertools
from collections import Counter


def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()
def load_data_and_labels(positive_data_file, negative_data_file):
    """
    Loads MR polarity data from files, splits the data into words and generates labels.
    Returns split sentences and labels.
    """
    # Load data from files
    positive_examples = list(open(positive_data_file,encoding='utf-8').readlines())
    positive_examples = [s.strip() for s in positive_examples]
    negative_examples = list(open(negative_data_file,encoding='utf-8').readlines())
    negative_examples = [s.strip() for s in negative_examples]
    # Split by words
    x_text = positive_examples + negative_examples
    x_text = [clean_str(sent) for sent in x_text]
    # Generate labels
    positive_labels = [[0, 1] for _ in positive_examples]
    negative_labels = [[1, 0] for _ in negative_examples]
    y = np.concatenate([positive_labels, negative_labels], 0)
    return [x_text, y]
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
    yield shuffled_data[start_index:end_index]

In [3]:
# Parameters
# ==================================================

# Data loading params
tf.flags.DEFINE_float("dev_sample_percentage", 0.1, "Percentage of the training data to use for validation")
tf.flags.DEFINE_string("positive_data_file", "data/train_pos_full.txt", "Data source for the positive data.")
tf.flags.DEFINE_string("negative_data_file", "data/train_neg_full.txt", "Data source for the positive data.")

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularization lambda (default: 0.0)")

# Training parameters
tf.flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 200, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")


# Load data
print("Loading data...")
x_text, y = load_data_and_labels(FLAGS.positive_data_file, FLAGS.negative_data_file)

# Build vocabulary
max_document_length = max([len(x.split(" ")) for x in x_text])
vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
x = np.array(list(vocab_processor.fit_transform(x_text)))

# Randomly shuffle data
np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(y)))
x_shuffled = x[shuffle_indices]
y_shuffled = y[shuffle_indices]

# Split train/test set
# TODO: This is very crude, should use cross-validation
dev_sample_index = -1 * int(FLAGS.dev_sample_percentage * float(len(y)))
x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]
print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))

# Training
# ==================================================

with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        cnn = TextCNN(
            sequence_length=x_train.shape[1],
            num_classes=y_train.shape[1],
            vocab_size=len(vocab_processor.vocabulary_),
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            l2_reg_lambda=FLAGS.l2_reg_lambda)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.histogram_summary("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.scalar_summary("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.merge_summary(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.scalar_summary("loss", cnn.loss)
        acc_summary = tf.scalar_summary("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.merge_summary([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.train.SummaryWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.merge_summary([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.train.SummaryWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables())

        # Write vocabulary
        vocab_processor.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)

        # Generate batches
        batches = batch_iter(
            list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(x_dev, y_dev, writer=dev_summary_writer)
                print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))


Parameters:
ALLOW_SOFT_PLACEMENT=True
BATCH_SIZE=64
CHECKPOINT_EVERY=100
DEV_SAMPLE_PERCENTAGE=0.1
DROPOUT_KEEP_PROB=0.5
EMBEDDING_DIM=128
EVALUATE_EVERY=100
FILTER_SIZES=3,4,5
L2_REG_LAMBDA=0.0
LOG_DEVICE_PLACEMENT=False
NEGATIVE_DATA_FILE=data/train_neg_full.txt
NUM_EPOCHS=200
NUM_FILTERS=128
POSITIVE_DATA_FILE=data/train_pos_full.txt
UNROLLED_LSTM=False

Loading data...
Vocabulary Size: 438252
Train/Dev split: 1800000/200000
Instructions for updating:
Please switch to tf.summary.histogram. Note that tf.summary.histogram uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on their scope.
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
Instruc

In [8]:
# Parameters
# ==================================================

# Eval Parameters
#tf.flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
#tf.flags.DEFINE_string("checkpoint_dir", "data/test_data", "Checkpoint directory from training run")


FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")

# CHANGE THIS: Load data. Load your own data here
if FLAGS.eval_train:
    x_raw, y_test = data_helpers.load_data_and_labels(FLAGS.positive_data_file, FLAGS.negative_data_file)
    y_test = np.argmax(y_test, axis=1)
else:
    x_raw = ["a masterpiece four years in the making", "everything is off."]
    y_test = [1, 0]

# Map data into vocabulary
vocab_path = os.path.join(FLAGS.checkpoint_dir, "..", "vocab")
vocab_processor = learn.preprocessing.VocabularyProcessor.restore(vocab_path)
x_test = np.array(list(vocab_processor.transform(x_raw)))

print("\nEvaluating...\n")

# Evaluation
# ==================================================
checkpoint_file = tf.train.latest_checkpoint(FLAGS.checkpoint_dir)
graph = tf.Graph()
with graph.as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        # Load the saved meta graph and restore variables
        saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
        saver.restore(sess, checkpoint_file)

        # Get the placeholders from the graph by name
        input_x = graph.get_operation_by_name("input_x").outputs[0]
        # input_y = graph.get_operation_by_name("input_y").outputs[0]
        dropout_keep_prob = graph.get_operation_by_name("dropout_keep_prob").outputs[0]

        # Tensors we want to evaluate
        predictions = graph.get_operation_by_name("output/predictions").outputs[0]

        # Generate batches for one epoch
        batches = data_helpers.batch_iter(list(x_test), FLAGS.batch_size, 1, shuffle=False)

        # Collect the predictions here
        all_predictions = []

        for x_test_batch in batches:
            batch_predictions = sess.run(predictions, {input_x: x_test_batch, dropout_keep_prob: 1.0})
            all_predictions = np.concatenate([all_predictions, batch_predictions])

# Print accuracy if y_test is defined
if y_test is not None:
    correct_predictions = float(sum(all_predictions == y_test))
    print("Total number of test examples: {}".format(len(y_test)))
    print("Accuracy: {:g}".format(correct_predictions/float(len(y_test))))

# Save the evaluation to a csv
predictions_human_readable = np.column_stack((np.array(x_raw), all_predictions))
out_path = os.path.join(FLAGS.checkpoint_dir, "..", "prediction.csv")
print("Saving evaluation to {0}".format(out_path))
with open(out_path, 'w') as f:
    csv.writer(f).writerows(predictions_human_readable)


Parameters:
ALLOW_SOFT_PLACEMENT=True
BATCH_SIZE=64
CHECKPOINT_DIR=data/test_data
CHECKPOINT_EVERY=100
DEV_SAMPLE_PERCENTAGE=0.1
DROPOUT_KEEP_PROB=0.5
EMBEDDING_DIM=128
EVALUATE_EVERY=100
FILTER_SIZES=3,4,5
L2_REG_LAMBDA=0.0
LOG_DEVICE_PLACEMENT=False
NEGATIVE_DATA_FILE=data/train_neg_full.txt
NUM_EPOCHS=200
NUM_FILTERS=128
POSITIVE_DATA_FILE=data/train_pos_full.txt
UNROLLED_LSTM=False



AttributeError: eval_train